In [69]:
import os
import cv2
import pickle
import base64
import csv
import numpy as np
import requests
import pandas as pd

train_dir = r'Cars Dataset\train'
test_dir = r'Cars Dataset\test'
Dataset = r'Cars Dataset'

In [70]:
data_set = os.listdir(Dataset)
print("List file in Cars Dataset:", data_set)

# Show all list file in test
test_files = os.listdir(test_dir)
print("List file in test:", test_files)

# Show all list file in train
train_files = os.listdir(train_dir)
print("List file in train:", train_files)

List file in Cars Dataset: ['test', 'train']
List file in test: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']
List file in train: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


In [71]:
brands  =[
              "Audi",
              "Hyundai Creta",
              "Mahindra Scorpio",
              'Rolls Royce',
              'Swift',
              'Tata Safari',
              'Toyota Innova'
         ]

In [72]:
def img2Hog(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    url = 'http://localhost:8080/api/gethog'
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url, json={"img":data})
    
    return response.json()

In [73]:
def getHog(path_dir):
    response = []
    Amount = 0
    for filebrand in os.listdir(path_dir):
        count =0
        for img_filename in os.listdir(path_dir + '/' + filebrand):
            img_file_name = path_dir + '/' + filebrand + "/" + img_filename
            img = cv2.imread(img_file_name)
            res = img2Hog(img)
            hog = list(res["hog"])
            hog.append(brands.index(filebrand))
            response.append(hog)
            count = count + 1
        print(str((filebrand))+': have "'+str(count)+'" cars.')
        Amount = Amount + count
    print('All amount: ',Amount)    
    return response

In [74]:
list_train = getHog(train_dir)

Audi: have "814" cars.
Hyundai Creta: have "271" cars.
Mahindra Scorpio: have "316" cars.
Rolls Royce: have "311" cars.
Swift: have "424" cars.
Tata Safari: have "441" cars.
Toyota Innova: have "775" cars.
All amount:  3352


In [75]:
list_test = getHog(test_dir)

Audi: have "199" cars.
Hyundai Creta: have "67" cars.
Mahindra Scorpio: have "75" cars.
Rolls Royce: have "74" cars.
Swift: have "102" cars.
Tata Safari: have "106" cars.
Toyota Innova: have "190" cars.
All amount:  813


In [76]:
# save data to pkl function
def save_pkl(fileNames, dataPath):
    pkl_filename = fileNames + ".pkl"
    
    with open(pkl_filename, "wb") as pkl_file:
        pickle.dump(dataPath, pkl_file)
    print("data preparation is done")
        
# load data to pkl function 
def load_pkl(fileName):
    listDatas = pickle.load(open(fileName + '.pkl','rb'))
    return listDatas

In [77]:
# Call function savePkl and send train path folder
save_pkl('train_cars',list_train)


data preparation is done


In [78]:
# Call function savePkl and send test path folder
save_pkl('test_cars',list_test)

data preparation is done


In [79]:
# Send train_cars.pkl file and call function load_pkl
dataset_train = load_pkl('train_cars')
print("Data train : ",len(dataset_train))

Data train :  3352


In [80]:
# Send test_cars.pkl file and call function load_pkl
dataset_test = load_pkl('test_cars')
print("Data test : ",len(dataset_test))

Data test :  813


In [81]:
# Make listdata train become to array
train_arr = np.array(dataset_train)
x_train = train_arr[:,0:-1]
y_train = train_arr[:,-1]


In [82]:
# Make listdata test become to array
test_arr = np.array(dataset_test)
x_test = test_arr[:,0:-1]
y_test = test_arr[:,-1]

In [83]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Train data with Decision Tree
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)

DecisionTreeClassifier()

In [84]:
# Check how good data is it 
y_pred = dtc.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 48.708487084870846
Confusion Matrix : 
 [[ 79  21  13  15  24  21  26]
 [  9  37   4   5   3   2   7]
 [ 11   5  26   5   5   9  14]
 [ 23   6   8  15   6   7   9]
 [  9  10   5   5  53   2  18]
 [ 16   3   3   5   5  66   8]
 [ 24  15   4   9  12   6 120]]


In [85]:
# Save model file with pickle
path_model ='carbrandmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))